# Gaussian Mixture Model & Expectation - Maximization Algorithm




In [20]:
from ipysketch import Sketch

In [21]:
sk = Sketch('gmm')

In [23]:
sk

Button(description='Edit', style=ButtonStyle())

Output()

![gmm](../contents/gmm.PNG)

## Variables explanations

* $\pi_k$: probability of grabbing a sample from distribution $k$ (in this case Gaussian $k$)
* $N(\vec{x}_n | \mu_k, \Sigma_k)$: given Gaussian distribution parameters mean ($\mu_k$), and covariance ($\Sigma_k$), the probability of getting a sample result $\vec{x}_n$
* $\gamma_{nk}$: Given that we have got a sample $\vec{x}_n$, the probability that this sample comes from distribution $k$. (Note this is derived from Bayes rule: $\gamma_{nk} := p(z_k | \vec{x}_n) \propto p(\vec{x}_n | z_k)\cdot p(z_k) = N(\vec{x}_n | \mu_k, \Sigma_k) \cdot \pi_k$
* $T_n$: probability of grabbing a sample result $\vec{x}_n$ from the whole population
* $p(\vec{X})$: probability of achieving a consecutive results $(\vec{x}_1, \vec{x}_2, ..., \vec{x}_N)$ (since we assume each experiment is independent), this term is called **likelihood**
* $\ln p(\vec{X})$: the **log-likelihood**
* $N_k$: probability that the consecutive results $(\vec{x}_1, \vec{x}_2, ..., \vec{x}_N)$ come from distribution $k$
* $(\pi_k^*, \mu_k^*, \Sigma_k^*)$: updated parameters

## 3 steps of expectation-maximization algorithm (iterative method)

1. In *Initialization step*,  we must initialise our parameters $(\pi_k, \mu_k, \Sigma_k)$. In this case, we are going to use the results of KMeans as an initial value for $\mu_k$, set $\pi_k$ to one over the number of clusters and $\Sigma_k$ to the identity matrix. We could also use random numbers for everything, but using a sensible initialisation procedure will help the algorithm achieve better results.

2. In *Expectation step*, we calculate the matrix elements $\gamma_{nk}$. This is the *expected values* from our guess of the parameters $(\pi_k, \mu_k, \Sigma_k)$.

3. In *Maximization step*, we update the parameters to $(\pi_k^*, \mu_k^*, \Sigma_k^*)$




## Ref

1. [知乎：【机器学习】EM——期望最大（非常详细）](https://zhuanlan.zhihu.com/p/78311644)
2. [GMM Implementation](https://github.com/ocontreras309/ML_Notebooks/blob/master/GMM_Implementation.ipynb)
3. [Gaussian Mixture Models Explained](https://towardsdatascience.com/gaussian-mixture-models-explained-6986aaf5a95)